In [90]:
from dash import Dash, html, dcc, Input, Output, callback
import plotly.express as px
import pandas as pd

In [91]:
df1 = pd.read_csv('data/daily_sales_data_0.csv')
df2 = pd.read_csv('data/daily_sales_data_1.csv')
df3 = pd.read_csv('data/daily_sales_data_2.csv')

# Concatenate the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)
df = df[df['product'] == 'pink morsel']

In [92]:
# Calculate the total sales
df['price'] = df['price'].str.replace('$', '').astype(float)
df['sales'] = df['price'] * df['quantity']

In [93]:
df = df.drop(columns=['price', 'quantity', 'product'])

In [94]:
df.to_csv('data/pink_morsel_sales.csv', index=False)

In [95]:
# Define the center date
center_date = pd.to_datetime('2021-01-15')
df['date'] = pd.to_datetime(df['date'])

# Filter ±7 days
mask = (df['date'] >= center_date - pd.Timedelta(days=7)) & (df['date'] <= center_date + pd.Timedelta(days=7))
df_filtered = df[mask]

In [96]:
app = Dash()
fig = px.line(
    df_filtered,
    x='date',
    y='sales',
    color='region',  # This creates one line per region!
    title='Sales Over Time by Region'
)

In [98]:
# App layout
app.layout = html.Div(
    children=[
        html.H1(
            'Sales Dashboard',
            style={
                'textAlign': 'center',
                'color': COLORS["secondary"],
                'marginBottom': '20px'
            }
        ),

        html.Div([
            html.Label('Filter by region:', style={'fontWeight': 'bold', 'color': COLORS["accent"]}),
            dcc.RadioItems(
                ['north', 'south', 'east', 'west', 'All'],
                'All',
                id='region-filter',
                inline=True,
                style={'marginTop': '10px'},
                labelStyle={'margin-right': '15px'}
            )
        ], style={'textAlign': 'center', 'marginBottom': '30px'}),

        dcc.Graph(
            id='sales-line-graph',
            style={'height': '70vh'}
        )
    ],
    style={
        'backgroundColor': COLORS["primary"],
        'padding': '30px',
        'fontFamily': 'Arial, sans-serif'
    }
)


# Callback to update figure
@app.callback(
    Output('sales-line-graph', 'figure'),
    Input('region-filter', 'value')
)
def update_graph(selected_region):
    if selected_region == 'All':
        filtered_df_region = df_filtered
    else:
        filtered_df_region = df_filtered[df_filtered['region'] == selected_region]

    fig = px.line(
        filtered_df_region,
        x='date',
        y='sales',
        color='region',
        title=f'Sales Over Time by Region: {selected_region.title()}',
        markers=True
    )

    fig.update_layout(
        plot_bgcolor=COLORS["background"],
        paper_bgcolor=COLORS["primary"],
        font_color=COLORS["text"],
        title_font_size=22,
        title_x=0.5
    )
    return fig


# Run app
if __name__ == '__main__':
    app.run(debug=True)